In [1]:
!nvidia-smi

Tue Mar  7 09:49:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:67:03.0 Off |                  Off |
| N/A   34C    P0    27W / 250W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install transformers SentencePiece torch tqdm


Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


In [5]:
!pip install -U scikit-learn

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 9.8 MB 22.2 MB/s eta 0:00:01
     |████████████████████████████████| 33.8 MB 54.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.7 MB 50.0 MB/s eta 0:00:01
     |████████████████████████████████| 31.2 MB 62.8 MB/s eta 0:00:01     |███████████████████████████████ | 30.2 MB 62.8 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 32.1 MB/s eta 0:00:01


In [6]:
import math
from tqdm import tqdm
import numpy as np
from transformers import MT5ForConditionalGeneration, T5Tokenizer
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

In [7]:
class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        #它的构造函数需要传入一个原始的transformer word embedding（wte），
        #n_tokens表示任务需要的标记数量，默认为10
        #random_range表示初始化embedding时使用的随机范围，默认为0.5，
        #initialize_from_vocab表示是否从默认的vocab中初始化embedding，默认为True
        #    这个类用来给模型附加一个用于学习的embedding
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        #这行代码的作用是将 initialize_embedding 方法返回的 tensor 转化为 nn.Parameter，并将其赋值给 self.learned_embedding。
        #nn.parameter.Parameter 是 nn.Parameter 的别名，用于将一个 tensor 转化为一个可优化的参数。在 PyTorch 中，需要优化的参数都需要被包装为 nn.Parameter 类型。
      #nn.parameter.Parameter 的输入可以是任意 shape 的 tensor，它会被包装为一个 nn.Parameter 对象，并保存在一个 module 中。
    #在这个 module 中，这个 nn.Parameter 对象可以像其他 PyTorch 的 tensor 一样使用，但它还可以被注册到 optimizer 中，从而被优化器更新它的值。
    #通常情况下，nn.Embedding 的嵌入向量是随机初始化的，并不是根据特定的预先训练好的词向量来初始化的。但是，也可以将预训练好的词向量加载到 nn.Embedding 中，以便更好地初始化嵌入向量。
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                                  n_tokens, 
                                                                                  random_range, 
                                                                                initialize_from_vocab))
 #(n_tokens, embedding_size)，其中 embedding_size 是原始 Transformer word embedding 的维度（通常是 768 或 1024 等）。
#该 tensor 的每一行表示一个 token 的 embedding。           
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
         # 有两种初始化方式，一种是从预训练模型copy一部分token，进行训练
        # 另一种是随机生成一部分训练
        # 结果上来说区别不大
 #如果 initialize_from_vocab 为 True，则会从原始的 Transformer word embedding 中复制前 n_tokens 个 token 对应的 embedding 作为 self.learned_embedding 的初始值。
        #这里使用 .clone().detach() 的原因是为了避免共享原始 embedding 的权重，从而使得 self.learned_embedding 的值独立于原始 embedding 的值
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
#如果 initialize_from_vocab 为 False，则会随机生成一个形状为 (n_tokens, embedding_size) 的 tensor 作为 self.learned_embedding 的初始值，
#其中 tensor 中每个元素的值都在 [-random_range, random_range] 的范围内随机生成。
#这种初始化方式通常用于在没有预训练模型的情况下训练自定义任务。
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
#self.wte 是一个 nn.Embedding 类的实例，它接收一个大小为 (batch_size, sequence_length) 的张量作为输入，其中每个元素都是一个代表单词在词表中索引的整数。
#tokens 张量的形状为 (batch_size, sequence_length)        
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
         # 把我们新加入的固定长度的，用于代表任务的prompt embedding，和实际的embedding合并
        return torch.cat([learned_embedding, input_embedding], 1)

In [8]:
#zh-dataset-inews 是一个用于中文文本分类任务的数据集，包含了10个类别的新闻文本，每个类别有大约3000条数据。这个数据集在国内的一些NLP竞赛中被广泛使用。
!pip install zh-dataset-inews

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 11.3 MB 69 kB/s eta 0:00:01     |█████████                       | 3.2 MB 1.6 MB/s eta 0:00:06


In [9]:
#这行代码引入了 zh_dataset_inews 数据集中的训练集、验证集和测试集数据。
#title_train 是一个列表，包含了训练集中所有新闻的标题，label_train 是一个列表，包含了训练集中所有新闻的标签。
#title_dev 和 label_dev 分别是验证集中的新闻标题和标签，title_test 和 label_test 分别是测试集中的新闻标题和标签。
from zh_dataset_inews import title_train, label_train, title_dev, label_dev, title_test, label_test

In [10]:
def generate_data(batch_size, n_tokens, title_data, label_data):
#定义了一个labels列表，其中存储了三个Tensor，用于将标签转换为模型所需的形式。
    labels = [
        torch.tensor([[3]]),  # \x00
        torch.tensor([[4]]),  # \x01
        torch.tensor([[5]]),  # \x02
    ]

    def yield_data(x_batch, y_batch, l_batch):
# 对 x_batch 进行 padding 操作，将不同长度的文本填充到相同长度。
#x_batch 中的每个元素都是一个 token id 列表，通过 padding 后，这些列表会组成一个形状为 (batch_size, max_len) 的张量 x，
#其中 max_len 是 x_batch 中最长的 token id 列表的长度。 如果 batch_first=True，则 x 的第一个维度是 batch size，否则 x 的第二个维度是 batch size。
        x = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
    #y_batch 是一个列表，包含了当前 batch 中所有数据的标签，dim=0 意味着沿着第 0 维进行拼接，也就是将每个标签数据按照行的方向拼接在一起。
        y = torch.cat(y_batch, dim=0)
#这一行代码是创建一个 mask tensor，这个 tensor 的维度和输入的 x tensor 是相同的，但是它的元素都是 0 或 1。对于 x tensor 中的每个元素，
#如果它是一个 padding token，那么它对应的 mask tensor 中的元素就是 0，否则就是 1
        m = (x > 0).to(torch.float32) 
#这行代码是创建一个形状为 (batch_size, n_tokens) 的张量，其中所有的元素都被填充为数字 1。这个张量会被作为解码器的输入，用于生成模型的输出
        decoder_input_ids = torch.full((x.size(0), n_tokens), 1)
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
            m = m.cuda()
            decoder_input_ids = decoder_input_ids.cuda()
        return x, y, m, decoder_input_ids, l_batch

    x_batch, y_batch, l_batch = [], [], []
    for x, y in zip(title_data, label_data):
        context = x
#首先将其文本内容 context 传递给 tokenizer 进行分词和编码，得到一个 PyTorch 张量 inputs。其中 return_tensors="pt" 表示 tokenizer 函数会返回一个 PyTorch 张量。
        inputs = tokenizer(context, return_tensors="pt")
#这行代码将一个值为1的张量拼接在每个样本的输入标记之前，即在输入标记的最前面添加一个初始标记（start token）
#这个 start token 可以被看做是一个类似于语言模型中的起始标记，用于指示生成序列的起点。
        inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)
        l_batch.append(y)
        y = labels[y]
        y = torch.cat([torch.full((1, n_tokens - 1), -100), y], 1)
        x_batch.append(inputs['input_ids'][0])
        y_batch.append(y)
#这段代码是为了生成一个batch的数据。如果当前数据集中的数据量达到了指定的batch size，就返回一个batch的数据，然后将 x_batch、y_batch 和 l_batch 清空，
#以便下一个batch的数据填充。如果当前数据集中的数据量不足一个batch size，那么就不返回任何数据。
        if len(x_batch) >= batch_size:
            yield yield_data(x_batch, y_batch, l_batch) #label_batch
            x_batch, y_batch, l_batch = [], [], []

    if len(x_batch) > 0:
        yield yield_data(x_batch, y_batch, l_batch)
        x_batch, y_batch, l_batch = [], [], []

In [31]:
torch.full((1, n_tokens - 1), -100).shape # -100

torch.Size([1, 99])

In [38]:
torch.tensor([2, 2]).unsqueeze(0)

tensor([[2, 2]])

In [37]:
torch.cat([torch.full((1, n_tokens - 1), -100), torch.tensor([2, 2]).unsqueeze(0)], 1)

tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100,    2,    2]])

In [11]:
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
n_tokens = 100
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=True)
#model.set_input_embeddings 是一个 PyTorch 函数，用于设置模型的输入嵌入。
#在这个函数中，我们将输入嵌入替换为我们自己的 SoftEmbedding 类，以便能够根据模型的需要动态地生成嵌入向量。
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.cuda()

In [14]:
parameters = list(model.parameters())
for x in parameters[1:]:  # frozen
    x.requires_grad = False

In [15]:
parameters[0]

Parameter containing:
tensor([[ 1.7500e+00, -1.6719e+00,  2.4062e+00,  ...,  6.9580e-03,
         -9.8828e-01, -4.6875e-01],
        [ 8.5625e+00,  5.5625e+00, -1.7109e+00,  ...,  7.7812e+00,
         -5.2812e+00, -3.2188e+00],
        [ 6.8750e-01, -4.5312e-01,  5.7812e-01,  ...,  7.3828e-01,
         -3.0078e-01,  2.0312e-01],
        ...,
        [-4.9219e-01,  1.9141e-01, -4.3555e-01,  ..., -8.0469e-01,
         -4.3359e-01,  5.8594e-01],
        [ 4.9609e-01,  1.1797e+00,  3.7109e-01,  ...,  1.7090e-01,
         -2.5195e-01, -3.3789e-01],
        [ 1.6328e+00,  3.4961e-01,  3.9062e-01,  ...,  1.9336e-01,
         -7.4219e-01,  3.1836e-01]], device='cuda:0', requires_grad=True)

In [16]:
parameters[2]

Parameter containing:
tensor([[-1.3977e-02,  3.8818e-02,  5.7129e-02,  ...,  4.9316e-02,
         -8.1177e-03, -3.8147e-03],
        [ 6.3965e-02, -1.0193e-02, -2.0020e-02,  ..., -8.3618e-03,
         -1.1902e-02, -2.6978e-02],
        [-1.6357e-02, -4.4922e-02,  4.8584e-02,  ..., -1.6479e-02,
         -4.0039e-02,  6.3782e-03],
        ...,
        [ 7.7820e-03, -6.5918e-03, -3.9062e-03,  ...,  1.9165e-02,
          7.4863e-05, -2.6001e-02],
        [-1.4587e-02,  1.8433e-02, -2.6489e-02,  ..., -3.9062e-02,
         -4.0527e-02,  4.1992e-02],
        [ 7.8125e-02,  1.6602e-02,  6.4941e-02,  ...,  4.2152e-04,
          4.5166e-02, -1.1780e-02]], device='cuda:0')

In [43]:
for x, y, m, dii, true_labels in generate_data(8, n_tokens, title_train, label_train):
    assert dii.shape == y.shape
    print(x.shape)
    print("-----------------------------------------------------------------------")
    print(m.shape)
    print("-----------------------------------------------------------------------")
    print(dii.shape)
    print("-----------------------------------------------------------------------")
    print(true_labels)
    print("-----------------------------------------------------------------------")
    print(y.shape)
    outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
    assert outputs['logits'].shape[:2] == y.shape
#这行代码的作用是从模型的输出 outputs 中取出最后一个时刻的输出结果 logits，然后从中选择第3到第5个元素（即3,4,5）作为标签预测的结果，再使用 .argmax(-1) 取出每个样本中最大的概率对应的下标，
#然后使用 .detach().cpu().numpy().tolist() 将结果转化为 Python 的列表并返回。
    pred_labels = outputs['logits'][:, -1, 3:6].argmax(-1).detach().cpu().numpy().tolist()
    break

torch.Size([8, 125])
-----------------------------------------------------------------------
torch.Size([8, 125])
-----------------------------------------------------------------------
torch.Size([8, 100])
-----------------------------------------------------------------------
[1, 1, 0, 2, 2, 2, 1, 2]
-----------------------------------------------------------------------
torch.Size([8, 100])


In [44]:
batch_size = 16
n_epoch = 50
total_batch = math.ceil(len(title_train) / batch_size)
dev_total_batch = math.ceil(len(title_dev) / batch_size)
use_ce_loss = True
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(s_wte.parameters(), lr=0.5)

for epoch in range(n_epoch):
    print('epoch', epoch)

    all_true_labels = []
    all_pred_labels = []
    losses = []
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, title_train, label_train)), total=total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        
        optimizer.zero_grad()
#decoder_input_ids 在 T5 模型中是作为解码器输入的 token id 序列，是一个形状为 (batch_size, sequence_length) 的张量，其中 batch_size 表示输入数据的批大小
#，sequence_length 表示 token 序列的长度。在该张量中，每个位置上的值代表着该位置对应的 token 的 id。
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        pred_labels = outputs['logits'][:, -1, 3:6].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels

        if use_ce_loss:
            logits = outputs['logits'][:, -1, 3:6]
            true_labels_tensor = torch.tensor(true_labels, dtype=torch.long).cuda()
            loss = ce_loss(logits, true_labels_tensor)
        else:
            loss = outputs.loss
        loss.backward()
        optimizer.step()
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        losses.append(loss_value)

        acc = accuracy_score(all_true_labels, all_pred_labels)
        pbar.set_description(f'train: loss={np.mean(losses):.4f}, acc={acc:.4f}')

    all_true_labels = []
    all_pred_labels = []
    losses = []
    with torch.no_grad():
        pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, title_dev, label_dev)), total=dev_total_batch)
        for i, (x, y, m, dii, true_labels) in pbar:
            all_true_labels += true_labels
            outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
            loss = outputs.loss
            loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
            losses.append(loss_value)
            pred_labels = outputs['logits'][:, -1, 3:6].argmax(-1).detach().cpu().numpy().tolist()
            all_pred_labels += pred_labels
            acc = accuracy_score(all_true_labels, all_pred_labels)
            pbar.set_description(f'dev: loss={np.mean(losses):.4f}, acc={acc:.4f}')

epoch 0


dev: loss=0.8667, acc=0.4925: 100%|██████████| 63/63 [00:09<00:00,  6.63it/s]


epoch 1


train: loss=0.0601, acc=0.4986:  94%|█████████▍| 315/335 [01:40<00:06,  3.14it/s]


KeyboardInterrupt: 

In [20]:
parameters2 = list(model.parameters())

In [21]:
!nvidia-smi

Tue Mar  7 12:28:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:67:03.0 Off |                  Off |
| N/A   37C    P0    37W / 250W |  18130MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
parameters2[0]

Parameter containing:
tensor([[-12.1830, -30.1978, -20.4185,  ..., -27.8833,  15.5076,   8.6759],
        [-18.3980,  27.7653, -24.2388,  ...,  10.0886,  15.9386,  -1.8228],
        [-15.5309,   1.9090,   4.1854,  ..., -15.4622,   7.6856,   9.5103],
        ...,
        [ 32.2267,  -8.5860, -27.1125,  ..., -16.3302, -17.3018,  -6.5299],
        [ 10.1303,   1.3383, -12.2403,  ...,  16.5293,  -9.3344,  -0.4445],
        [  7.7325,  32.6109,  -3.6557,  ...,  22.8400,   9.9415,   9.7184]],
       device='cuda:0', requires_grad=True)

In [23]:
parameters2[2]

Parameter containing:
tensor([[-1.3977e-02,  3.8818e-02,  5.7129e-02,  ...,  4.9316e-02,
         -8.1177e-03, -3.8147e-03],
        [ 6.3965e-02, -1.0193e-02, -2.0020e-02,  ..., -8.3618e-03,
         -1.1902e-02, -2.6978e-02],
        [-1.6357e-02, -4.4922e-02,  4.8584e-02,  ..., -1.6479e-02,
         -4.0039e-02,  6.3782e-03],
        ...,
        [ 7.7820e-03, -6.5918e-03, -3.9062e-03,  ...,  1.9165e-02,
          7.4863e-05, -2.6001e-02],
        [-1.4587e-02,  1.8433e-02, -2.6489e-02,  ..., -3.9062e-02,
         -4.0527e-02,  4.1992e-02],
        [ 7.8125e-02,  1.6602e-02,  6.4941e-02,  ...,  4.2152e-04,
          4.5166e-02, -1.1780e-02]], device='cuda:0')

In [24]:
def predict(text):
    inputs = tokenizer(text, return_tensors='pt')
    inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)

    decoder_input_ids = torch.full((1, n_tokens), 1)
    with torch.no_grad():
        outputs = model(input_ids=inputs['input_ids'].cuda(), decoder_input_ids=decoder_input_ids.cuda())
    logits = outputs['logits'][:, -1, 3:6]
    pred = logits.argmax(-1).detach().cpu().numpy()[0]
    # print(logits)
    return pred

In [25]:
train_rets = []
for i in tqdm(range(len(title_train))):
    pred = predict(title_train[i])
    train_rets.append((label_train[i], pred, title_train[i]))

100%|██████████| 5355/5355 [04:49<00:00, 18.48it/s]


In [26]:
rets = []
for i in tqdm(range(len(title_test))):
    pred = predict(title_test[i])
    rets.append((label_test[i], pred, title_test[i]))

100%|██████████| 999/999 [00:52<00:00, 19.07it/s]


In [27]:
print(
    accuracy_score(
        [x[0] for x in train_rets],
        [x[1] for x in train_rets],
    )
)

0.5017740429505135


In [28]:
print(
    accuracy_score(
        [x[0] for x in rets],
        [x[1] for x in rets],
    )
)

0.4994994994994995


In [29]:
print(
    accuracy_score(
        [x[0] for x in rets],
        [0] * len(rets),
    ),
    accuracy_score(
        [x[0] for x in rets],
        [1] * len(rets),
    ),
    accuracy_score(
        [x[0] for x in rets],
        [2] * len(rets),
    )
)

0.0990990990990991 0.4944944944944945 0.4064064064064064
